In [1]:
import numpy as np

from anony_process_pipeline import *
from src.modules.utils_image import pillow_image_to_bytes, numpy_image_to_pillow
from database_controller import DatabaseController


Configuration chargée et vérifiée.


In [2]:
from werkzeug.datastructures import FileStorage
import os

def import_subject_images(ids):
    image_folder = "../../data/yalefaces"
    sujet = f"subject0{ids}_"
    image_files = []
    for filename in os.listdir(image_folder):
        if filename.startswith(sujet):
            chemin_fichier = f"{image_folder}/{filename}"
            file = open(chemin_fichier, "rb")
            file_storage = FileStorage(stream=file, filename=filename)
            image_files.append(file_storage)
    return image_files

# Import images
img_list = import_subject_images(3)

# Print result
img_list

[<FileStorage: 'subject03_centerlight.png' (None)>,
 <FileStorage: 'subject03_glasses.png' (None)>,
 <FileStorage: 'subject03_happy.png' (None)>,
 <FileStorage: 'subject03_leftlight.png' (None)>,
 <FileStorage: 'subject03_noglasses.png' (None)>,
 <FileStorage: 'subject03_normal.png' (None)>,
 <FileStorage: 'subject03_rightlight.png' (None)>,
 <FileStorage: 'subject03_sad.png' (None)>,
 <FileStorage: 'subject03_sleepy.png' (None)>,
 <FileStorage: 'subject03_surprised.png' (None)>,
 <FileStorage: 'subject03_wink.png' (None)>]

In [3]:
# ---------------------------------------
# Étape 1 : Preprocessing (Simplifiée)
# ---------------------------------------
image_size = (50, 50)
res1, image_size = run_preprocessing(filestorage_list=img_list, image_size_override=image_size)

# IMG IMPORT ENDING: Realise images in storage ###
for img_in_storage in img_list: ##################
    img_in_storage.close() #######################
##################################################

# Print result
res1

2025-04-08 12:08:57,862 - INFO - Exécution du pré-traitement standard avec IMAGE_SIZE=(50, 50)...
2025-04-08 12:08:57,863 - INFO - Traitement de 11 fichiers FileStorage.
Preprocessing (FileStorage List): 100%|██████████| 11/11 [00:00<00:00, 422.50it/s]
2025-04-08 12:08:57,893 - INFO - Pré-traitement terminé. 11 images traitées pour 1 sujets.


{'upload_subject_1': [{'resized_image': <PIL.Image.Image image mode=RGB size=50x50>,
   'grayscale_image': <PIL.Image.Image image mode=L size=50x50>,
   'normalized_image': array([[0.92156863, 0.92156863, 0.92156863, ..., 1.        , 1.        ,
           1.        ],
          [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
           1.        ],
          [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
           1.        ],
          ...,
          [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
           1.        ],
          [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
           1.        ],
          [0.88627451, 0.88627451, 0.88627451, ..., 0.88627451, 0.88627451,
           0.88627451]]),
   'imageId': 'upload_img_0'},
  {'resized_image': <PIL.Image.Image image mode=RGB size=50x50>,
   'grayscale_image': <PIL.Image.Image image mode=L size=50x50>,
   'normalized_image': array([[0.92156863, 0.9215686

In [4]:
# ---------------------------------------
# Étape 2 : K-Same-Pixel (Nouvelle Fonction Modulaire)
# ---------------------------------------
k_same_k_value = 4
res2 = run_k_same_anonymization(res1, k_same_k_value)

# Print result
res2
# [img['flattened_anonymized_image'] for img in res2["upload_subject_1"]]

2025-04-08 12:09:08,142 - INFO - Application de K-Same-Pixel avec k=4...
K-Same Pixel: 100%|██████████| 1/1 [00:00<00:00, 997.46it/s]
2025-04-08 12:09:08,146 - INFO - Traitement K-Same-Pixel terminé.


{'upload_subject_1': [{'flattened_anonymized_image': array([0.92156863, 0.92156863, 0.92156863, ..., 0.69411767, 0.69411767,
          0.69411767], dtype=float32),
   'imageId': 'upload_img_0'},
  {'flattened_anonymized_image': array([0.92156863, 0.92156863, 0.92156863, ..., 0.8862745 , 0.8862745 ,
          0.8862745 ], dtype=float32),
   'imageId': 'upload_img_1'},
  {'flattened_anonymized_image': array([0.92156863, 0.92156863, 0.92156863, ..., 0.8862745 , 0.8862745 ,
          0.8862745 ], dtype=float32),
   'imageId': 'upload_img_2'},
  {'flattened_anonymized_image': array([0.92156863, 0.92156863, 0.92156863, ..., 0.69411767, 0.69411767,
          0.69411767], dtype=float32),
   'imageId': 'upload_img_3'},
  {'flattened_anonymized_image': array([0.92156863, 0.92156863, 0.92156863, ..., 0.8862745 , 0.8862745 ,
          0.8862745 ], dtype=float32),
   'imageId': 'upload_img_4'},
  {'flattened_anonymized_image': array([0.92156863, 0.92156863, 0.92156863, ..., 0.8862745 , 0.8862745 ,


In [5]:
# ---------------------------------------
# Étape 3 : PEEP Eigenface
# ---------------------------------------

flattened_stack_list = [img['flattened_anonymized_image'] for img in res2["upload_subject_1"]]
flattened_stack_np = np.array(flattened_stack_list, dtype=np.float32)

n_samples, n_features = flattened_stack_np.shape
n_components_ratio = 0.8
n_components = min(max(1, int(n_components_ratio * n_samples)), n_features)

res3 = run_eigenface(flattened_stack_np, n_components)

# Print result
pca, mean_face, projection = res3 # Eingenface a disparue...
print("n_components:", n_components)
mean_face

2025-04-08 12:09:10,947 - ERROR - Erreur PCA: cannot reshape array of size 2500 into shape (100,100). Stack shape: (11, 2500), n_components: 8


n_components: 8


In [12]:
# Étape 4 : Ajout de bruit différentiel
epsilon = 0.01
res4 = run_add_noise(projection, epsilon, sensitivity=1.0)

# Étape 5 : Reconstruction
res5 = run_reconstruction(pca, res4, image_size)

# Print result
import base64
from io import BytesIO
from PIL import Image
imgs = [Image.open(BytesIO(base64.b64decode(img))) for img in res5]
print(len(imgs))
imgs[0].show()
res5

11


['...']

In [13]:
db = DatabaseController(r"..\..\data\database.db")

user_id = db.add_user(np.array(res5))
print(f"user_id: {user_id}")

user = db.get_user(user_id)

base64_str = user[0]
image_bytes = base64.b64decode(base64_str)
image = Image.open(io.BytesIO(image_bytes))
image.show()

Create db connection at: D:\myDocs\Apprentissage\Isep\Year_3\Brno-Privacy_Preserving_Face_Recognition_Project\data\database.db
user_id: 10
